# Llama PIRO

This notebook implements the PIRO prompting techinique on the Llama

##  Importing necessary libraries

Ensure that the Python environment you are running this in has all the libraries present in [requirements.txt](requirements.txt).


In [4]:
%%capture
!pip install colab-xterm

In [5]:
%load_ext colabxterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%%capture
%pip install -r requirements_llama_piro.txt

In [8]:
%%capture
!curl https://ollama.ai/install.sh | sh  # install ollama
!apt-get install lshw
%autoreload 2

To start using ollama, run the following on Xterm terminal:

```ollama serve```

In [ ]:
#Launch the xterm terminal
%xterm

Download the speciifc model you want to use

The following notebook is tested on 2 models :
1.  llama2
2.  vicuna

In [ ]:
# Pull the model you want to use
!ollama pull llama2   # !ollama pull <Your desired Model>

In [11]:
#Importing the necessary libraries
import langchain
import openai
import json
import os
import time
import pandas as pd

from litellm import completion
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import AgentExecutor

## OpenAI API Key
Set your OpenAI API Key as an environment variable using:

```
%env OPENAI_API_KEY = <Your API Key>
```

In [12]:
os.environ["OPENAI_API_KEY"]= "YOUR_OPENAI_API_KEY"  # Set your OpenAI API key

## Loading the API documentation and usage examples

The required datasets [.json] -
- API/Tool Descriptions (api_desc.json)
- Tool Usage Example (examples.json)
- Queries (PS_queries.json)
  
  These can be loaded either by
  1. Uploading the datasets on google drive and using its id to use `gdown` method (as shown).
  2. By directly uploading the datasets to the runtime

In [ ]:
!gdown 1gG6Ghpkjxqz7vlPjCs2pTdmOSqjt0EIM
!gdown 1nJZpWHRdowbdzdRI7ylb1RrZUV_2E6k9
!gdown 1JCjW2f0fTsL6W7r7QjQO2FrwKNUj37ao

In [ ]:
documentation_file_path = '/content/api_desc.json'
with open(documentation_file_path, 'r') as file:
    documentation = json.load(file)


examples_file_path = '/content/examples.json'
with open(examples_file_path, 'r') as file:
    examples = json.load(file)


queries_file_path = '/content/PS_queries.json'
with open(queries_file_path, 'r') as file:
    queries = json.load(file)

print(documentation)
print(examples)
print(queries)

# Retrievers for relevant tools and examples

- The following section implements the retriever to filter out only the relevant tools and examples from the complete tool description bank and usage examples respectively.
- The `get_tools` function returns the set of relevant tools, and their argument on the basis of user query, similarily `get_examples` function returns the set of relevant tool usage examples.
- Uses the FAISS index to store the created embeddings


In [15]:
# Applying semantic search on the API descriptions
API_descriptions = [
    Document(page_content=t['description'], metadata={"index": i})
    for i, t in enumerate(documentation)
]
API_descriptions_vector_store = FAISS.from_documents(API_descriptions, OpenAIEmbeddings())
API_retriever = API_descriptions_vector_store.as_retriever()
def get_tools(query):
    docs = API_retriever.get_relevant_documents(query)
    tools = [documentation[d.metadata["index"]]['tool'] for d in docs]
    arguments = [documentation[d.metadata["index"]] for d in docs]
    return tools, arguments

In [16]:
#Applying semantic search on the API usage examples

API_usage_examples = [
    Document(page_content=t['Query'], metadata={"index": i})
    for i, t in enumerate(examples)
]
API_usage_examples_vector_score = FAISS.from_documents(API_usage_examples, OpenAIEmbeddings())
Examples_retriever = API_usage_examples_vector_score.as_retriever()
def get_examples(query):
    docs = Examples_retriever.get_relevant_documents(query)
    return [examples[d.metadata["index"]] for d in docs]

# PIRO Prompts

## Planning

In [17]:
def get_planning_prompt(query, tools):
    planning_template =f"""
    Given a query and a list of APIs with their arguments, find whether the APIs can solve the query or not. If not, return empty list. Else, extract values or variables from the query that correspond to each argument of the tools that needs to be called. If some argument value isn't known, use APIs that can retrieve those argument values. Return a JSON file containing the tools in the order they need to be called. Identify cases where an API argument depends on the output of a previous API call. Replace such arguments with the notation $PREV[i] where 'i' represents the order of the previous API call in the chain.

    Query: {query}
    APIs: {tools}
    Answer:
    """
    return planning_template

## Improvement
**Note:** Same template will be used for the reflection process also

In [18]:
def get_improvement_prompt(examples, query, solution):
    improvement_template = f"""
    Given a list of examples, each containing a query and the corresponding APIs to be called to solve the query, find whether the JSON present in current solution solves the current query.
    If yes, modify its format to match with that of solutions in examples and return the modified JSON.
    If not, modify the current JSON solution to include necessary tools from list of available tools.
    If no solution is available, return a blank list.

    Examples : {examples}

    Current Query : {query}
    Current Solution : {solution}

    Answer:
    """
    return improvement_template

## Optimization

In [19]:
def get_optimization_prompt(query, solution, tools):
    optimization_template = f"""
    Given a query, solution and available APIs, optimize the number of APIs calls in the solution. Do not use API calls unless absolutely necessary. Look for redundancy and check for mistakes in the combination of API calls. Return current solution if no changes are necessary.

    Current Query : {query}
    Current Solution : {solution}
    Available APIs : {tools}
    Final solution (same format as current solution):
    """
    return optimization_template

# Execution

In [20]:
#Creating empty dataframes to store the results
df2 = pd.DataFrame(columns=['query', 'f_output','time'])

In [21]:
def ask_llm(prompt):
    # Function to interact with the Llama model and get the response
    response = completion(
            model="ollama/llama2",
            messages = [{"content": prompt,"role": "user"}],
            api_base="http://localhost:11434"
    )
    return response.choices[0].message.content

The `get_result` function return the final predicted output,the total time taken to generate the output.

- We chose the top 3 Tool usage examples


In [22]:
def get_result(Query):
    start = time.time()

    _,Tools = get_tools(Query)
    Examples = get_examples(Query)[:3]
    planning_prompt = get_planning_prompt(Query, Tools)
    Solution = ask_llm(planning_prompt)
    print(f"Output 1: {Solution}")


    improvement_prompt = get_improvement_prompt(Examples, Query, Solution)
    Solution = ask_llm(improvement_prompt)
    print(f"Output 2: {Solution}")

    optimization_prompt = get_optimization_prompt(Query, Solution, Tools)
    Solution = ask_llm(optimization_prompt)
    print(f"Output 3: {Solution}")

    end = time.time()

    time_taken = end - start
    return Solution, time_taken

In [ ]:
#Running the pipeline on the queries
for query in queries:
  f_output ,time_taken = get_result(query)
  f_output = f_output.strip("```").lstrip("json")
  df2 = df2.append({'query': query, 'f_output': f_output, 'time': time_taken}, ignore_index=True)
  time.sleep(5)

In [ ]:
df2

In [ ]:
#Exporting the results to a xlsx file
df2.to_excel('PS_results.xlsx')